In [1]:
import torch
import pandas as pd

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from langchain_community.llms       import LlamaCpp, CTransformers
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.embeddings import LlamaCppEmbeddings, HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma, FAISS
from langchain_community.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader, WebBaseLoader, PyPDFDirectoryLoader, CSVLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.prompts       import PromptTemplate
from langchain.chains        import LLMChain, RetrievalQA

In [2]:
data = load_dataset("HuggingFaceTB/cosmopedia", "stories", split="train[:100]")
num_examples = len(data)

data = data.to_pandas()
data.to_csv("E:/models/huggingface.cache/HuggingFaceTB___cosmopedia/stories/dataset.csv")
print(num_examples)

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/43 [00:00<?, ?it/s]

100


In [3]:
loader = CSVLoader(file_path='E:/models/huggingface.cache/HuggingFaceTB___cosmopedia/stories/dataset.csv')
data = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(data)

In [5]:
modelPath = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
 model_name=modelPath, 
 model_kwargs=model_kwargs, 
 encode_kwargs=encode_kwargs 
)

In [6]:
db = FAISS.from_documents(docs, embeddings)
# db.save_local("E:/models/huggingface.cache/HuggingFaceTB___cosmopedia/stories/faiss_index_constitution")
# vectorstore.save_local("faiss_index_constitution")

#     # Load from local storage
#     persisted_vectorstore = FAISS.load_local("faiss_index_constitution", embeddings)

#     # Use RetrievalQA chain for orchestration
#     qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=persisted_vectorstore.as_retriever())
#     result = qa.run(query)
#     print(result)

In [7]:
model_name = "E:/models/2b-gemma"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

d:\Python312\Lib\site-packages\accelerate\utils\modeling.py:780: UserWarning: expandable_segments not supported on this platform (Triggered internally at ..\c10/cuda/CUDAAllocatorConfig.h:30.)
  _ = torch.tensor([0], device=i)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
from transformers import pipeline
import torch

pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    return_tensors='pt',
    max_length=512,
    max_new_tokens=512,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda"
)

In [10]:
llm = HuggingFacePipeline(
 pipeline=pipe,
 model_kwargs={"temperature": 0.7, "max_length": 512},
)

In [11]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

In [12]:
qa.invoke("Write an educational story for young children.")

Both `max_new_tokens` (=512) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
A:\transformers\src\transformers\models\gemma\modeling_gemma.py:554: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


KeyError: 'generated_text'